In [1]:
import os
import logging
import wandb
import torch

from pathlib import Path
from lightning.pytorch import (
    callbacks,
    loggers,
    Trainer,
    utilities
)

from model import MNIST
from data_module import MNISTDataModule
from extentions.callbacks import DiffEarlyStopping, EarlyStopping

/home/.cache/pypoetry/virtualenvs/digits-k5qqIq61-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [2]:
logging.getLogger("lightning.pytorch").setLevel(logging.INFO)
root_path = Path('../')
os.environ['WANDB_NOTEBOOK_NAME'] = "train.ipynb"

dm = MNISTDataModule(data_dir=(root_path / 'data').as_posix())

# Training

In [3]:
model = MNIST(
    optimizer_name='Adam',
    # optimizer_hparams={
    #     'lr': 0.001,
    #     'momentum': 0.9
    # }
)

earlystopping_callbacks = [
    DiffEarlyStopping(
        monitor1="val_loss",
        monitor2="train_loss",
        diff_threshold=0.05, # like val_loss=0.09, train_loss=0.04
        patience=5,
        verbose=True
    ),
    EarlyStopping(
        monitor="val_acc",
        min_delta=0.0,
        mode='max',
        stopping_threshold=99.99,
        patience=5,
        verbose=True
    ),
]

checkpoint_callback = callbacks.ModelCheckpoint(
    filename="epoch={epoch}-loss={val_loss:.3f}",
    auto_insert_metric_name=False,
    monitor='val_loss',
    mode='min',
    save_top_k=3
)

In [4]:
utilities.model_summary.ModelSummary(model) # type: ignore

/home/.cache/pypoetry/virtualenvs/digits-k5qqIq61-py3.10/lib/python3.10/site-packages/lightning/pytorch/utilities/model_summary/model_summary.py:415: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  warning_cache.warn(


  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 0     
-------------------------------------
0         Trainable params
0         Non-trainable params
0         Total params
0.000     Total estimated model params size (MB)

In [5]:
log_dir = root_path/'logs'
log_dir.mkdir(exist_ok=True)
logger = loggers.WandbLogger(  # type: ignore
    project='MNIST',
    save_dir=log_dir,
    log_model='all',
)

max_time =  {'minutes': 20} if torch.cuda.is_available() else {'hours': 2}
trainer = Trainer(
    min_epochs=10,
    max_epochs=50,
    log_every_n_steps=1,
    max_time=max_time,
    logger=logger,
    callbacks=[checkpoint_callback] + earlystopping_callbacks, # type: ignore
    enable_model_summary=False,
)

wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, datamodule=dm)
wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: 0it [00:00, ?it/s]

/home/.cache/pypoetry/virtualenvs/digits-k5qqIq61-py3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 7, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

EarlyStopping: Monitored metric val_acc did not improve in the last 5 records. Best score: 99.039. Signaling Trainer to stop.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train_acc_epoch,▁▆▆▇▇▇▇▇▇█████████████
train_acc_step,▁▃▃▁▃▆▆▁██▃▆▃██▆█████████████▃█████▆████
train_loss_epoch,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▇▄▆▅▂▃▆▂▂▂▂▇▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁
trainer/global_step,▁▁▁▁▂▁▂▂▃▃▂▃▂▄▂▄▂▄▄▅▅▅▅▃▆▃▆▃▆▆▇▇▇▇▄█▄█▄▄
val_acc_epoch,▁▃▄▅▆▇▆▆█▇▇██▇▇▆██████
val_acc_step,▆█▆▁▃███▆▆██▆▆▆█▆█▆▆███▆▆███▆█▆▆███▆██▆█
val_loss_epoch,█▆▄▃▃▂▂▃▁▂▂▂▁▂▂▅▂▁▂▂▂▂
val_loss_step,▄▂▃▅▄▂▁▁▃▄▁▁▃▂▃▂▂▁▂█▁▁▁▃▆▁▁▂▄▁▅▂▁▁▁▂▁▁▂▁
epoch,21
